In [87]:
import pandas as pd

trainning_df = pd.read_pickle('../data/training_classif_data.pkl')
df_with_nan = pd.read_pickle('../data/df_with_nan.pkl')


In [88]:
# Standardiser les données ????

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# Standardiser toutes les colonnes du DataFrame

# il faut encoder !! puis standardiser !!
# df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
# df.head()


In [89]:
# CLASSIFICATION 'Code_type_local'

# TRAINING

# Import train_test_split function
from sklearn.model_selection import train_test_split
# ########### Random forest
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Random forest mieux mais 8min de calcul


X = trainning_df.drop(columns=['Code_type_local', 'Commune', 'Section', 'No_plan', 'Nature_culture',
                               'Departement', 'Code_postal', 'day', 'month', 'year', 'latitude', 'longitude', 'Surface_terrain'])
                      

y = trainning_df['Code_type_local']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1)


modele_decision_tree = DecisionTreeClassifier(random_state=42)

modele_decision_tree.fit(X_train, y_train)
y_pred = modele_decision_tree.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9318270354499416


In [90]:
df_with_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46639 entries, 0 to 47190
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   No_disposition             46639 non-null  int64  
 1   Valeur_fonciere            46639 non-null  float64
 2   Code_postal                46639 non-null  object 
 3   Commune                    46639 non-null  object 
 4   Section                    46639 non-null  object 
 5   No_plan                    46639 non-null  int64  
 6   Nombre_de_lots             46639 non-null  int64  
 7   Code_type_local            24727 non-null  float64
 8   Surface_reelle_bati        24688 non-null  float64
 9   Nombre_pieces_principales  24688 non-null  float64
 10  Nature_culture             38989 non-null  object 
 11  Surface_terrain            38989 non-null  float64
 12  day                        46639 non-null  object 
 13  month                      46639 non-null  obj

In [91]:
# Filling Nans 

df_with_nan['Nombre_pieces_principales'] = df_with_nan['Nombre_pieces_principales'].fillna(
    df_with_nan['Nombre_pieces_principales'].median())
df_with_nan['Surface_reelle_bati'] = df_with_nan['Surface_reelle_bati'].fillna(
    df_with_nan['Surface_reelle_bati'].median())


In [92]:
# Use previous model to fill 'Code_type_local' == NaN

# Réimporter les données de base, tous ceux qui ont Code_type_local == NaN, puis appliquer le modèle le classif 

# Sélectionnez les individus avec Code_type_local manquant
individus_code_type_local_manquants = df_with_nan[df_with_nan['Code_type_local'].isna()]
#print(individus_code_type_local_manquants)

X_manquants = individus_code_type_local_manquants.drop(columns=['Code_type_local', 'Commune', 'Section', 'No_plan', 'Nature_culture',
                                                                'Departement', 'Code_postal', 'day', 'month', 'year', 'latitude', 'longitude', 'Surface_terrain'])
#print(X_manquants)


y = individus_code_type_local_manquants['Code_type_local']

# Prédisez Code_type_local pour les individus manquants
predictions_code_type_local = modele_decision_tree.predict(X_manquants)

# Remplacez les valeurs manquantes dans le DataFrame original avec les prédictions
df_with_nan.loc[df_with_nan['Code_type_local'].isna(), 'Code_type_local'] = predictions_code_type_local


print(len(df_with_nan))
df_with_nan.head()


46639


,No_disposition,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,day,month,year,latitude,longitude,Departement
0,1,182500.0,3100,MONTLUCON,CH,396,0,4.0,75.0,0.0,S,307.0,12,9,2018,46.338588,2.603905,03
1,2,2500.0,3100,MONTLUCON,BO,458,0,2.0,79.0,3.0,S,75.0,14,2,2018,46.338588,2.603905,03
2,1,138100.0,4400,BARCELONNETTE,AD,513,1,2.0,50.0,2.0,NaN,NaN,29,10,2018,44.378561,6.652151,04
3,1,27000.0,3380,LA CHAPELAUDE,ZW,127,0,1.0,48.0,2.0,S,350.0,2,2,2018,46.427637,2.504336,03
4,1,196550.0,3700,BRUGHEAS,YB,24,0,1.0,116.0,5.0,AG,514.0,25,9,2018,46.072835,3.366060,03


In [93]:

df = df_with_nan



In [94]:
# # Replace last empty variables

# median_surface_by_type = df.groupby('Code_type_local')[
#     'Surface_reelle_bati'].median()
# # Remplacez les NaN dans "Surface_reelle_bat" par la médiane correspondante basée sur "Code_type_local"
# df['Surface_reelle_bati'] = df['Surface_reelle_bati'].fillna(
#     df['Code_type_local'].map(median_surface_by_type))

# median_surface_by_type = df.groupby('Code_type_local')[
#     'Nombre_pieces_principales'].median()
# # Remplacez les NaN dans "Surface_reelle_bat" par la médiane correspondante basée sur "Code_type_local"
# df['Nombre_pieces_principales'] = df['Nombre_pieces_principales'].fillna(
#     df['Code_type_local'].map(median_surface_by_type))


# Removing useless variables
df = df.drop(columns=["Nature_culture", "Surface_terrain"])

In [95]:
pourcentages_nan = (df.isna().sum() / len(df)) * 100
print(pourcentages_nan)
print(len(df))

df.head()

No_disposition               0.0
Valeur_fonciere              0.0
Code_postal                  0.0
Commune                      0.0
Section                      0.0
No_plan                      0.0
Nombre_de_lots               0.0
Code_type_local              0.0
Surface_reelle_bati          0.0
Nombre_pieces_principales    0.0
day                          0.0
month                        0.0
year                         0.0
latitude                     0.0
longitude                    0.0
Departement                  0.0
dtype: float64
46639


,No_disposition,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,day,month,year,latitude,longitude,Departement
0,1,182500.0,3100,MONTLUCON,CH,396,0,4.0,75.0,0.0,12,9,2018,46.338588,2.603905,03
1,2,2500.0,3100,MONTLUCON,BO,458,0,2.0,79.0,3.0,14,2,2018,46.338588,2.603905,03
2,1,138100.0,4400,BARCELONNETTE,AD,513,1,2.0,50.0,2.0,29,10,2018,44.378561,6.652151,04
3,1,27000.0,3380,LA CHAPELAUDE,ZW,127,0,1.0,48.0,2.0,2,2,2018,46.427637,2.504336,03
4,1,196550.0,3700,BRUGHEAS,YB,24,0,1.0,116.0,5.0,25,9,2018,46.072835,3.366060,03


In [96]:
# Adding Variable 'Prix_moyen_m2'

import numpy as np

# Groupez les données par 'Commune' et calculez la somme de 'Surface_reel' et 'Valeur_fonciere' pour chaque commune
groupe_commune = df.groupby('Commune').agg(
    {'Surface_reelle_bati': 'sum', 'Valeur_fonciere': 'sum'})

# Calculez le prix moyen au mètre carré par commune
groupe_commune['Prix_moyen_m2'] = groupe_commune['Valeur_fonciere'] / \
    groupe_commune['Surface_reelle_bati']


# Fusionnez le DataFrame original avec le DataFrame groupe_commune sur la colonne 'Commune'
df = pd.merge(df, groupe_commune[['Prix_moyen_m2']],
              left_on='Commune', right_index=True, how='left')



# Remplace les inf par NaN
df = df.replace([np.inf, -np.inf], np.nan)
# Supprime les lignes avec NaN dans 'Prix_moyen_m2'
df = df.dropna(subset=['Prix_moyen_m2'], how='any')

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 46636 entries, 0 to 47190
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   No_disposition             46636 non-null  int64  
 1   Valeur_fonciere            46636 non-null  float64
 2   Code_postal                46636 non-null  int64  
 3   Commune                    46636 non-null  object 
 4   Section                    46636 non-null  object 
 5   No_plan                    46636 non-null  int64  
 6   Nombre_de_lots             46636 non-null  int64  
 7   Code_type_local            46636 non-null  float64
 8   Surface_reelle_bati        46636 non-null  float64
 9   Nombre_pieces_principales  46636 non-null  float64
 10  day                        46636 non-null  int64  
 11  month                      46636 non-null  int64  
 12  year                       46636 non-null  int64  
 13  latitude                   46636 non-null  flo

In [97]:
# Export final data

df.to_pickle('../data/final_data.pkl')
